<a href="https://colab.research.google.com/github/celikn/GEE_Repository/blob/main/GEE_Image_Export_20220217.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install earthengine-api
!pip install earthengine-api --upgrade


     |████████████████████████████████| 236 kB 5.1 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.299-py3-none-any.whl size=265523 sha256=c55883f462e4d785dd3f65c85809b96ee121180da950777a30ff00bf484313bb
  Stored in directory: /root/.cache/pip/wheels/8d/a1/f5/953e280e6fa406de38e445a4c7ab6fd0d7d39901b22d295104
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.298
    Uninstalling earthengine-api-0.1.298:
      Successfully uninstalled earthengine-api-0.1.298


In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
!pip install geetools
!pip install --upgrade geetools
!pip install geemap


     |████████████████████████████████| 74 kB 1.7 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=8d23412607d9ce8750bb0229ff31e34a2bc91987bd8087f4979ee706e1d58555
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 45.3 MB/s 
     |████████████████████████████████| 1.2 MB 47.9 MB/s 
     |████████████████████████████████| 216 kB 53.6 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 130 kB 46.8 MB/s 
     |████████████████████████████████| 5.4 MB 38.5 MB/s 
     |████████████████████████████████| 1.7 MB 47.1 MB/s 
     |████████████████████████████████| 8.5 MB 46.0 MB/s 
     |████████████████████████████████| 176 kB 71.3 MB/s 
     |███████████████

In [ ]:
### SENTINEL 
## Image'i goruntuleme, Ankarada bir lokasyona denk gelen nokta.
import geemap
import ee

lon,lat =32.717443, 39.824942 ## Ankara

Map = geemap.Map(center=[lon,lat], zoom=9) ## Haritayi olusturur. 
aoi = ee.Geometry.Point(lon,lat) ## filtrede kullanmak uzere bir nokta olusturuyoruz. Cakisan imageleri belirlemek icin. 

##Sentinel_2A verisinin collection icinde tarih ve nokta ile cakisan ve cloud pixel %5 dan az olan imagelerin ortalamasini alir. 
se2= ee.ImageCollection('COPERNICUS/S2')\
.filterDate("2019-01-01","2019-12-31")\
.filterBounds(aoi)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
##.select(["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"])

se2_median=se2.median() ## haritada goruntulemek icin median image olusturuyoruz. 

## Harita da gormek icin rgb bandlarini visualize ediyoruz. 
rgb = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":3000,"bands":rgb}
Map.centerObject(aoi, 7)
Map.addLayer(se2_median, rgbViz, "S2")
Map

Map(center=[39.824942, 32.717443], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [ ]:
############# Calisan Ornek Sentinel Imageleri Tiff olarak Google Drive'a indirme ################################
## Read More : https://github.com/gee-community/gee_tools
import geetools
images_info= se2.aggregate_array('system:index').getInfo()
print(images_info)
print("image sayisi:"+str(len(images_info)))

data_type = 'float32'
name_pattern = 'S2SR_{system_date}'
date_pattern = 'yMMdd' # dd: day, MMM: month (JAN), y: year
scale = 10
folder_name = 'SentinelImages_Ankara'

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=se2,
            folder=folder_name,
            namePattern=name_pattern,
            scale=scale,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10)

In [ ]:
### LANDSAT 
## Image'i goruntuleme, Ankarada bir lokasyona denk gelen nokta.
import geemap
import ee

lon,lat =32.717443, 39.824942 ## Ankara

Map = geemap.Map(center=[lon,lat], zoom=9) ## Haritayi olusturur. 
aoi = ee.Geometry.Point(lon,lat) ## filtrede kullanmak uzere bir nokta olusturuyoruz. Cakisan imageleri belirlemek icin. 

##Sentinel_2A verisinin collection icinde tarih ve nokta ile cakisan ve cloud pixel %5 dan az olan imagelerin ortalamasini alir. 
la8= ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(aoi)\
.filter(ee.Filter.lt('CLOUD_COVER', 5))\
.filterDate("2019-10-01","2019-12-31")


la8_first=la8.median() ## haritada goruntulemek icin median image olusturuyoruz. 

## Harita da gormek icin rgb bandlarini visualize ediyoruz. 
rgb = ['B3', 'B2', 'B1']
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}
Map.centerObject(aoi, 7)
Map.addLayer(la8_first, rgbViz, "Landsat 8")
Map

Map(center=[39.824942, 32.717443], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [ ]:
############# Calisan Ornek Landsat Imageleri Tiff olarak Google Drive'a indirme ################################
## Read More : https://github.com/gee-community/gee_tools
import geetools

imagesMetadata=la8.toList(la8.size()).getInfo()  ## collection listeye cevirip getInfo methodunu uyguluyor. 
imagesName =[val[1]["properties"]['system:index'] for val in enumerate(imagesMetadata)]  ## gelen metadata verilerinden image index 

print(imagesName)
print("image sayisi:"+str(len(imagesName)))

data_type = 'float32'
name_pattern = 'L8SR_{system_date}'
date_pattern = 'yMMdd' # dd: day, MMM: month (JAN), y: year
scale = 30
folder_name = 'LandsatImages_Ankara'

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=la8,
            folder=folder_name,
            namePattern=name_pattern,
            scale=scale,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10)

['LC08_177032_20191014', 'LC08_177032_20191030', 'LC08_177033_20191014', 'LC08_177033_20191030', 'LC08_178032_20191106']
image sayisi:5
exporting L8SR_20191014 to folder 'LandsatImages_Ankara' in GDrive
exporting L8SR_20191030 to folder 'LandsatImages_Ankara' in GDrive
exporting L8SR_20191014 to folder 'LandsatImages_Ankara' in GDrive
exporting L8SR_20191030 to folder 'LandsatImages_Ankara' in GDrive
exporting L8SR_20191106 to folder 'LandsatImages_Ankara' in GDrive


In [ ]:
### LANDSAT 
## Image'i goruntuleme, Ankarada bir lokasyona denk gelen nokta.
import geemap
import ee

lon,lat =32.717443, 39.824942 ## Ankara

Map = geemap.Map(center=[lon,lat], zoom=9) ## Haritayi olusturur. 
aoi = ee.Geometry.Point(lon,lat) ## filtrede kullanmak uzere bir nokta olusturuyoruz. Cakisan imageleri belirlemek icin. 

##Sentinel_2A verisinin collection icinde tarih ve nokta ile cakisan ve cloud pixel %5 dan az olan imagelerin ortalamasini alir. 
la7= ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(aoi)\
.filterDate("2000-10-01","2000-12-31")


la7_first=la7.median() ## haritada goruntulemek icin median image olusturuyoruz. 

## Harita da gormek icin rgb bandlarini visualize ediyoruz. 
rgb = ['SR_B3', 'SR_B2', 'SR_B1']
rgbViz = {"min":5000, "max":50000,"bands":rgb}
Map.centerObject(aoi, 7)
Map.addLayer(la7_first, rgbViz, "Landsat 7")
Map

Map(center=[39.824942, 32.717443], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [ ]:
############# Calisan Ornek Landsat Imageleri Tiff olarak Google Drive'a indirme ################################
## Read More : https://github.com/gee-community/gee_tools
import geetools

imagesMetadata=la7.toList(la8.size()).getInfo()  ## collection listeye cevirip getInfo methodunu uyguluyor. 
imagesName =[val[1]["properties"]['system:index'] for val in enumerate(imagesMetadata)]  ## gelen metadata verilerinden image index 

print(imagesName)
print("image sayisi:"+str(len(imagesName)))

data_type = 'float32'
name_pattern = 'L7SR_{system_date}'
date_pattern = 'yMMdd' # dd: day, MMM: month (JAN), y: year
scale = 30
folder_name = 'LandsatImages_Ankara'

tasks = geetools.batch.Export.imagecollection.toDrive(
            collection=la7,
            folder=folder_name,
            namePattern=name_pattern,
            scale=scale,
            datePattern=date_pattern,
            verbose=True,
            maxPixels=1e10)

['LE07_177032_20001001', 'LE07_177032_20001017', 'LE07_177032_20001102', 'LE07_177032_20001204', 'LE07_177032_20001220']
image sayisi:5
exporting L7SR_20001001 to folder 'LandsatImages_Ankara' in GDrive
exporting L7SR_20001017 to folder 'LandsatImages_Ankara' in GDrive
exporting L7SR_20001102 to folder 'LandsatImages_Ankara' in GDrive
exporting L7SR_20001204 to folder 'LandsatImages_Ankara' in GDrive
exporting L7SR_20001220 to folder 'LandsatImages_Ankara' in GDrive


In [ ]:
## Ekstra : Administrative alanlara gore filtreleme

lon,lat =32.717443, 39.824942 ## Ankara
Map = geemap.Map(center=[lon,lat], zoom=9) ## Haritayi olusturur. 

#### administrative sinirlari kullanarak filtreleme 
s2 = ee.ImageCollection("COPERNICUS/S2")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
admin1 =ee.FeatureCollection("FAO/GAUL/2015/level1")

administrativeArea = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Etimesgut'))
##administrativeArea = admin1.filter(ee.Filter.eq('ADM1_NAME', 'Ankara'))
aoi_geometry = administrativeArea.geometry()
s2= ee.ImageCollection('COPERNICUS/S2')\
.filterDate("2018-11-01","2019-12-31")\
.filterBounds(aoi)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) ## .select(["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"])

## CollectionInfo
images_info= s2.aggregate_array('system:index').getInfo()
print(images_info)
print("image sayisi:"+str(len(images_info)))

## Harita da gormek icin rgb bandlarini visualize ediyoruz. 
rgb = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":3000,"bands":rgb}
firstImageClipped = s2.first().clip(aoi_geometry)
Map.centerObject(aoi_geometry,12)
Map.addLayer(administrativeArea, {}, 'Etimesgut');
Map.addLayer(firstImageClipped, rgbViz, "S2")
Map

['20181103T084119_20181103T084639_T36TVK', '20181108T084141_20181108T084334_T36TVK', '20181208T084331_20181208T084330_T36TVK', '20190221T083959_20190221T083956_T36TVK', '20190308T083811_20190308T084001_T36TVK', '20190318T083701_20190318T083954_T36TVK', '20190422T083609_20190422T084547_T36TVK', '20190427T083601_20190427T083603_T36TVK', '20190517T083601_20190517T083604_T36TVK', '20190527T083601_20190527T084710_T36TVK', '20190606T083601_20190606T084745_T36TVK', '20190701T083609_20190701T084549_T36TVK', '20190721T083609_20190721T084751_T36TVK', '20190726T083611_20190726T083840_T36TVK', '20190731T083609_20190731T084610_T36TVK', '20190810T083609_20190810T084403_T36TVK', '20190825T083601_20190825T084818_T36TVK', '20190830T083609_20190830T084008_T36TVK', '20190904T083601_20190904T084432_T36TVK', '20190909T083559_20190909T084314_T36TVK', '20190914T083601_20190914T083726_T36TVK', '20190919T083619_20190919T084115_T36TVK', '20190929T083729_20190929T083853_T36TVK', '20191004T083811_20191004T084807_

Map(center=[39.907833856581554, 32.65090935545272], controls=(WidgetControl(options=['position', 'transparent_…